<h3>
    Crimes reported at Boston in 2018
</h3>
<p>
    In this project our purpose is to create a database and a schema using 
    Postgres where we will load the reported crimes happening in Boston area
    during calendar year 2018.
</p>
<p>
    After investigating the data types of the csv file we create an appropriate
    table for the storage of our data. Next, we create the groups
    with the appropriate priviliges (<b>readonly, readwrite </b>) and
    assign the created users on each group (inheriting the group's priviliges).
    Here we have to be careful as priviliges are inherited from higher groups,
    that said, every user in the <em>public</em> group can still have permission 
    to read/write in tables within public schema.
</p>
<p>
    A wonderful link describing the process can also be found at:
    <a href="https://aws.amazon.com/blogs/database/managing-postgresql-users-and-roles/">Managing PostgreSQL Users and Roles</a>
</p>

In [2]:
import psycopg2
import csv
import timeit
import numpy as np
import datetime

In [3]:
header = []
values_inspection = []
with open ('boston.csv', 'r') as file:
    reader = csv.reader(file)
    crime_values = [value for value in reader]
header.append(crime_values[0]) # inspection of the header 
values_inspection.append(crime_values[1]) # inspection of the data types of our file 
print(header,'\n' ,values_inspection)
#  usually upon loading of a file within another program some data types 
#  may change, therefore, it is always advisable to check the datatypes which 
#  we are working with (e.g. in this occasion the format of dd-mm--yyyy changed to
#  yyyyy--mm--dd)

[['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']] 
 [['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']]


In [4]:
def countable_set(name_of_file,column_index):  # create a function for parsing the file
    parsing_list = []                          # and outputting the elements of a column 
    with open(name_of_file, 'r') as file:
        next(file) #jump the header
        reader = csv.reader(file)
        for row in reader:
            parsing_list.append(row[column_index])
    return list(set(parsing_list)) # this is our unique_list

# In general a csv file may have as its columns a countable set of values (e.g. like days of the
# week). In this occasion we run a function to find all the unique values in that column and store 
# it in a list, call it unique_list. This will allow us to create a column constrained to receive
# only the values within the unique_list upon insertion of any new data.

In [5]:
length_of_row = len(crime_values[0])
for i in range(0,length_of_row):
    print(len(countable_set('boston.csv', i))) # check the number of unique elements within each 
                                               # column. Good candidates for enumerated types 
                                               # are columns with indices 1,2,4

298329
219
239
1177
7
18177
18177


In [6]:
offence_code = countable_set('boston.csv', 1)
description = countable_set('boston.csv', 2)
day_of_the_week = countable_set('boston.csv', 4)
print(offence_code, '\n\n', description, '\n\n', day_of_the_week, '\n\n')

['2010', '1602', '402', '2629', '2407', '3203', '3303', '522', '614', '1841', '3805', '2623', '1300', '2663', '3207', '2204', '2401', '629', '3110', '432', '530', '311', '1501', '3123', '3208', '2604', '562', '540', '3201', '3122', '315', '633', '3502', '3830', '2914', '2622', '1304', '2660', '670', '727', '541', '3302', '2662', '3002', '2007', '2664', '3620', '2619', '2900', '724', '1601', '770', '3006', '561', '1107', '3125', '3305', '616', '3007', '381', '3503', '624', '2647', '3004', '413', '2605', '123', '527', '1620', '637', '1845', '2003', '3109', '403', '2610', '1830', '615', '1415', '2609', '404', '900', '627', '3018', '2657', '112', '1402', '2641', '1844', '2910', '617', '3008', '1510', '2628', '1807', '639', '1863', '2613', '3820', '1866', '3803', '3114', '2646', '2905', '2617', '3501', '351', '802', '2612', '3016', '3001', '2102', '618', '520', '2648', '1806', '339', '1874', '1102', '1848', '2631', '2907', '3402', '3119', '3301', '1901', '3304', '1847', '349', '3106', '611'

In [14]:
# offence_code is just a 4-digit number so it's enough to store it as int of appropriate
# byte capacity 

length_description = [len(element) for element in description]
length_description.sort() # sort in terms of length, last element is the one with the largest 
                          # length
max_description_length = length_description[-1]  # this is the last element 

max_day_of_the_week = 0
for element in day_of_the_week:  # similarly we find the max length for 'day_of_the_week' 
    max_day_of_the_week = max(max_day_of_the_week, len(element))
print(max_day_of_the_week, '\t', max_description_length)

# the values of max_description_length and max_day_of_the_week could be the argunments for 
# a VARCHAR() type, in case we decide to store them as text of type VARCHAR()

9 	 58


In [28]:
conn = psycopg2.connect(dbname = 'postgres', user = 'postgres', password = 'durham17')
cur = conn.cursor()
conn.autocommit = True 
cur.execute('CREATE DATABASE crimes_db OWNER postgres')
conn.autocommit = False
conn.close()  

# First, we establish a connection to create a new database, since 
# in a transactional block commands are commited at once, we would
# like to commit the CREATE DATABASE option on its own, hence 
# we turned on the autocommit option.

In [7]:
conn = psycopg2.connect(dbname = 'crimes_db', user = 'postgres', password = 'durham17')
cur = conn.cursor()
cur.execute('CREATE SCHEMA crimes')


In [8]:
cur.execute('''  
                CREATE TYPE day_of_the_week AS ENUM 
                ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
            ''')

In [9]:
cur.execute('''
             CREATE TABLE crimes.boston_crimes(
                incidenent_number integer PRIMARY KEY,
                offence_codes smallint,
                desciption VARCHAR(58),
                date DATE,
                weekday day_of_the_week,
                lat real,
                long real
                
             )
            ''')

<p> 
   Our choices are based on the documentation of Postgres for its datatypes
   <a href='https://www.postgresql.org/docs/9.2/datatype-numeric.html'> Datatypes documentation </a>
</p>
        
<p>
    i.e. a small int is a datatype of 2 bytes,integer is datatype 0f 4 bytes, VARCHAR(58) takes into 
    account the maximum length of the entry
    desciption, which is 58;  DATE is the normal 
    format for date. Finally, real is a float datatype of 6 digits precision which for longtitude and 
    latitude is enough.
</p>

In [10]:
with open('boston.csv', 'r') as file:
    cur.copy_expert('COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;', file)

                                                       


<h3> Revoke all priviliges from public users </h3>

In [11]:
cur.execute('REVOKE ALL ON SCHEMA public FROM public') # revoke all priviliges from public 
                                                       # users on public schemas

cur.execute('REVOKE ALL ON DATABASE crimes_db FROM public') # no user from the public group 
                                                            # can connect to the database

<h3> Readonly group priviliges </h3>

In [12]:
cur.execute('CREATE GROUP readonly NOLOGIN;') 
cur.execute('GRANT CONNECT ON DATABASE crimes_db TO readonly')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly')
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly')
cur.execute('ALTER DEFAULT PRIVILEGES IN SCHEMA crimes GRANT SELECT ON TABLES TO readonly')
# the readonly group has access to the existing tables, therefore, to allow the group users
# read data from new tables within the schema we redefine the default priviliges in 
# Postgres for the specific schema

<h3> Readwrite group priviliges </h3>

In [13]:
cur.execute('CREATE GROUP readwrite NOLOGIN')
cur.execute('GRANT CONNECT ON DATABASE crimes_db TO readwrite')
cur.execute('GRANT USAGE, CREATE ON SCHEMA crimes TO readwrite')
cur.execute('GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite')
cur.execute('''
            ALTER DEFAULT PRIVILEGES IN SCHEMA crimes GRANT SELECT, INSERT, UPDATE, DELETE ON 
            TABLES TO readwrite;
            ''')

<h3> Create users </h3>

In [14]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'durham28&'" )
cur.execute('GRANT readonly TO data_analyst')
cur.execute("CREATE USER data_engineer WITH PASSWORD 'durham29&'")
cur.execute('GRANT readwrite TO data_engineer')

<h3> TEST </h3>

In [15]:
cur.execute('SELECT * FROM pg_user')
users = cur.fetchall()
all_users = [user for user in users]
print(all_users) #check the users 


[('postgres', 10, True, True, True, True, '********', None, None), ('data_analyst', 16607, False, False, False, False, '********', None, None), ('data_engineer', 16608, False, False, False, False, '********', None, None)]


In [16]:
cur.execute('''SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN  ('readwrite', 'readonly');
    ''')  

priviliges_readonly_readwrite = cur.fetchall()
print(priviliges_readonly_readwrite)  #check the user's priviliges
conn.close


[('readonly', 'SELECT'), ('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]


<function connection.close>

In [17]:
cur.execute('''
            SELECT 
                table_catalog,
                table_schema,
                table_name
            FROM information_schema.tables 
            WHERE table_schema = 'crimes'
            ''')

all_tables = cur.fetchall()
print(all_tables)  # check the schema design within the database  

[('crimes_db', 'crimes', 'boston_crimes')]


In [18]:
cur.execute('''
            SELECT * FROM crimes.boston_crimes
            LIMIT 10 
            ''') 

first_ten_rows = cur.fetchall()
print(first_ten_rows) #check the first 10 rows 

[(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', 42.35779, -71.13937), (2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', 42.30682, -71.0603), (3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', 42.34659, -71.072426), (4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', 42.334183, -71.07867), (5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', 42.275364, -71.09036), (6, 3820, 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', datetime.date(2018, 9, 3), 'Monday', 42.290195, -71.07159), (7, 724, 'AUTO THEFT', datetime.date(2018, 9, 3), 'Monday', 42.306072, -71.08273), (8, 3301, 'VERBAL DISPUTE', datetime.date(2018, 9, 3), 'Monday', 42.327015, -71.10555), (9, 301, 'ROBBERY - STREET', datetime.date(2018, 9, 3), 'Monday', 42.33152, -71.070854), (10, 3301, 'VERBAL DISPUTE', datetime.date(2018, 9, 3), 'Monday', 42.295147, -71.05861)]


In [19]:
cur.execute('''
            SELECT * FROM crimes.boston_crimes
            ORDER BY 1 DESC
            LIMIT 10 
            ''')
last_ten_rows = cur.fetchall()
print(last_ten_rows) #check the last 10 rows 

[(298329, 3125, 'WARRANT ARREST', datetime.date(2015, 6, 22), 'Monday', 42.33384, -71.08029), (298328, 3125, 'WARRANT ARREST', datetime.date(2016, 5, 31), 'Tuesday', 42.302334, -71.111565), (298327, 3125, 'WARRANT ARREST', datetime.date(2015, 7, 9), 'Thursday', 42.255928, -71.12317), (298326, 111, 'MURDER, NON-NEGLIGIENT MANSLAUGHTER', datetime.date(2015, 7, 9), 'Thursday', 42.255928, -71.12317), (298325, 3125, 'WARRANT ARREST', datetime.date(2016, 6, 5), 'Sunday', 42.336952, -71.08575), (298324, 3125, 'WARRANT ARREST', datetime.date(2018, 1, 27), 'Saturday', 42.322838, -71.10097), (298323, 1864, 'DRUGS - POSS CLASS D - INTENT MFR DIST DISP', datetime.date(2018, 1, 27), 'Saturday', 42.322838, -71.10097), (298322, 1849, 'DRUGS - POSS CLASS B - COCAINE, ETC.', datetime.date(2015, 8, 12), 'Wednesday', 42.35231, -71.063705), (298321, 1848, 'DRUGS - POSS CLASS B - INTENT TO MFR DIST DISP', datetime.date(2015, 8, 12), 'Wednesday', 42.35231, -71.063705), (298320, 3125, 'WARRANT ARREST', datet

In [20]:
conn.commit()
conn.close()